# Capstone Project - The Battle of Neighbourhoods

In [5]:
# install any required packages

!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge proj4
!conda install basemap
print('packages installed')

SyntaxError: invalid syntax (<ipython-input-5-2d8315dd9331>, line 5)

In [5]:
# import required libraries

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests
from bs4 import BeautifulSoup as bs
import lxml
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library
from folium import IFrame
from folium.features import DivIcon
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline

# import k-means from clustering stage
from sklearn.cluster import DBSCAN 
import sklearn.utils
from sklearn.preprocessing import StandardScaler 
from sklearn import metrics

print('import complete')

import complete


In [11]:
neighbourhoods = pd.read_csv("eastlondon_postcodes.csv")
print(neighbourhoods.shape)
neighbourhoods.head()

(29, 4)


,Postcode,Neighbourhood,Latitude,Longitude
0,SE1,"Bankside, South Bank, Lambeth (part), Southwar...",51.49960,-0.09613
1,SE1P,NaN,51.50642,-0.12721
2,SE2,"Abbey Wood, West Heath, Crossness, Thamesmead ...",51.49245,0.12127
3,SE3,"Blackheath, Kidbrooke, Westcombe Park",51.47138,0.02338
4,SE4,"Brockley, Crofton Park",51.46268,-0.03558


In [13]:
# Create a map of the neighbourhoods
address = 'London'          # get starting latitude and longitude for Glasgow
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates are {}, {}.'.format(latitude, longitude))

# create map using latitude and longitude values
neighbourhoodmap = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighbourhood in zip(neighbourhoods['Latitude'], neighbourhoods['Longitude'], neighbourhoods['Neighbourhood']):
    label = neighbourhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(neighbourhoodmap)  
    
neighbourhoodmap

C:\Users\jjwgo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinates are 51.5073219, -0.1276474.


In [8]:
# Define Foursquare Credentials and Version

CLIENT_ID = 'OKWEY3K21N5BZ3EYWCT3OBVTOX2APFWEU4RK1FNTDFRKDTOJ'          # Foursquare ID
CLIENT_SECRET = 'SRWCR3Y1DMHLUR4PYAUL1NQTZBM2GK1LU53XPWEZ25BZ1WJZ'          # Foursquare Secret
VERSION = '20191130'          # Foursquare API version
# uncomment before upload

In [9]:
# Define function to get x venues within y radius of each neighbourhood
def getVenues(names, latitudes, longitudes):
#    venues_list=[]
    venues_list = pd.DataFrame()
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # Define the corresponding URL
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            lat, 
            lng, 
            VERSION, 
            search_query, 
            radius, 
            LIMIT)
        # Send the GET Request
        results = requests.get(url).json()
        # assign relevant part of JSON to venues
        venues = results['response']['venues']
        # tranform venues into a dataframe
        venues_df = json_normalize(venues)
        # check results have been returned for neighbourhood
        if 'name' in venues_df.columns:
            #create 2nd dataframe with neighbourhood name, latitude and longitude for same length
            neigh_df = pd.DataFrame(index = range(0, len(venues_df)), columns = ['Neighbourhood', 'Latitude', 'Longitude'])
            neigh_df['Neighbourhood'] = name
            neigh_df['Latitude'] = lat
            neigh_df['Longitude'] = lng
            # keep only columns that include venue name, and anything that is associated with location
            required_cols = ['name', 'categories', 'location.lat', 'location.lng']
            filtered_df = venues_df.loc[:, required_cols]
            # function that extracts the category of the venue
            def get_cat_type(row):
                try:
                    cat_list = row['categories']
                except:
                    cat_list = row['venue.categories']
            
                if len(cat_list) == 0:
                    return None
                else:
                    return cat_list[0]['name']
            # filter the category for each row
            filtered_df['categories'] = filtered_df.apply(get_cat_type, axis=1)
        else:
            #create 2nd dataframe with neighbourhood name, latitude and longitude for same length
            neigh_df = pd.DataFrame(columns = ['Neighbourhood', 'Latitude', 'Longitude'])
            neigh_df.loc[0] = [name, lat, lng]
            # create a single row dataframe with correct column names but 'none' as the values
            filtered_df = pd.DataFrame([['none', 'none', 0, 0]], columns=list('ABCD'))
        # clean column names
        filtered_df.columns = ['Venue_Name', 'Venue_Cat', 'Venue_Lat', 'Venue_Lng']

        # merge two neighbourhood and venue dataframes
        joined_df = neigh_df.join(filtered_df, how = 'outer')

        venues_list = venues_list.append(joined_df).reset_index(drop=True)
    return(venues_list)

In [10]:
# define a query to search for specified venue type within specified metres
search_query = 'Coffee'
radius = 1500
LIMIT = 10000

# Call function
VenuesList1 = getVenues(names=neighbourhoods['Neighbourhood'],
                                   latitudes=neighbourhoods['Latitude'],
                                   longitudes=neighbourhoods['Longitude']
                                  )
print(VenuesList1.shape)

Bankside, South Bank, Lambeth (part), Southwark, Bermondsey (part), Vauxhall (part), Old Kent Road (part)
nan
Abbey Wood, West Heath, Crossness, Thamesmead (part), Plumstead (part)
Blackheath, Kidbrooke, Westcombe Park
Brockley, Crofton Park
Camberwell, Denmark Hill, Peckham, Brixton (part)
Catford, Bellingham, Hither Green (part)
Charlton
Deptford, Evelyn, Rotherhithe (part)
Eltham, Mottingham, New Eltham, Well Hall, Avery Hill (part), Falconwood (part), Sidcup (part), Chinbrook (part), Longlands (part) Kidbrooke (part), Shooter's Hill (part)
Greenwich, Maze Hill, Greenwich Peninsula
Kennington, Lambeth (part), Vauxhall (part), Oval (part)
Lee, Mottingham, Grove Park, Chinbrook, Hither Green (part), Eltham (part), Horn Park
Lewisham, Hither Green, Ladywell
New Cross
Peckham, Nunhead, South Bermondsey (part), Old Kent Road (part)
Rotherhithe (part), Surrey Quays, South Bermondsey (part)
Walworth, Kennington (part), Newington
Woolwich, Royal Arsenal, Plumstead, Shooter's Hill
Upper Norw

In [11]:
# define a query to search for specified venue type within specified metres
search_query = 'Restaurant'
radius = 750
LIMIT = 1000

# Call function
VenuesList2 = getVenues(names=neighbourhoods['Neighbourhood'],
                                   latitudes=neighbourhoods['Latitude'],
                                   longitudes=neighbourhoods['Longitude']
                                  )
print(VenuesList2.shape)

Bankside, South Bank, Lambeth (part), Southwark, Bermondsey (part), Vauxhall (part), Old Kent Road (part)
nan
Abbey Wood, West Heath, Crossness, Thamesmead (part), Plumstead (part)
Blackheath, Kidbrooke, Westcombe Park
Brockley, Crofton Park
Camberwell, Denmark Hill, Peckham, Brixton (part)
Catford, Bellingham, Hither Green (part)
Charlton
Deptford, Evelyn, Rotherhithe (part)
Eltham, Mottingham, New Eltham, Well Hall, Avery Hill (part), Falconwood (part), Sidcup (part), Chinbrook (part), Longlands (part) Kidbrooke (part), Shooter's Hill (part)
Greenwich, Maze Hill, Greenwich Peninsula
Kennington, Lambeth (part), Vauxhall (part), Oval (part)
Lee, Mottingham, Grove Park, Chinbrook, Hither Green (part), Eltham (part), Horn Park
Lewisham, Hither Green, Ladywell
New Cross
Peckham, Nunhead, South Bermondsey (part), Old Kent Road (part)
Rotherhithe (part), Surrey Quays, South Bermondsey (part)
Walworth, Kennington (part), Newington
Woolwich, Royal Arsenal, Plumstead, Shooter's Hill
Upper Norw

In [12]:
frames = [VenuesList1, VenuesList2]

VenuesList = pd.concat(frames)
VenuesList.to_csv('london_venues.csv')          #comment out once testing complete
VenuesList.sort_values('Neighbourhood', inplace=True)
VenuesList.head()

,Neighbourhood,Latitude,Longitude,Venue_Name,Venue_Cat,Venue_Lat,Venue_Lng
79,"Abbey Wood, West Heath, Crossness, Thamesmead ...",51.49245,0.12127,Taj Mahal Indian Restaurant,Indian Restaurant,51.491146,0.120691
100,"Abbey Wood, West Heath, Crossness, Thamesmead ...",51.49245,0.12127,none,none,0.000000,0.000000
347,"Anerley, Crystal Palace (part), Penge, Beckenh...",51.41009,-0.05683,Costa Coffee,Coffee Shop,51.420814,-0.067452
346,"Anerley, Crystal Palace (part), Penge, Beckenh...",51.41009,-0.05683,High Peak Coffee Co,Café,51.419183,-0.053644
345,"Anerley, Crystal Palace (part), Penge, Beckenh...",51.41009,-0.05683,Coffee Craft,Coffee Shop,51.400116,-0.075263


In [23]:
# During testing a CSV of VenuesList dataframe was created so as not to have to call foursquare multiple times
VenuesList = pd.read_csv('london_venues.csv')
VenuesList = VenuesList.drop(VenuesList.columns[0], axis = 1)
VenuesList[['Venue_Lat', 'Venue_Lng']] = VenuesList[['Venue_Lat', 'Venue_Lng']].astype('float')
print(VenuesList.shape)
VenuesList.sort_values('Neighbourhood', inplace=True)
VenuesList.head()

VenuesList.shape

(580, 7)


(580, 7)

In [24]:
# Create a map of the venues

# create map using latitude and longitude values
venuemap = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, venue in zip(VenuesList['Venue_Lat'], VenuesList['Venue_Lng'], VenuesList['Venue_Name']):
    label = venue
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(venuemap)  
    
venuemap

In [25]:
# Get counts of venues by neighbourhood
temp = VenuesList[~VenuesList['Venue_Name'].str.contains('none')].reset_index(drop=True)
value_counts = temp['Neighbourhood'].value_counts(dropna=True, sort=True)
temp = value_counts.rename_axis('Neighbourhood').reset_index(name='Counts')

results = pd.merge(neighbourhoods, temp, how='left', on='Neighbourhood')
results.fillna(0, inplace=True)

results.sort_values('Counts', ascending=False, inplace=True)
results

,Postcode,Neighbourhood,Latitude,Longitude,Counts
0,SE1,"Bankside, South Bank, Lambeth (part), Southwar...",51.49960,-0.09613,79.0
11,SE11,"Kennington, Lambeth (part), Vauxhall (part), O...",51.49084,-0.11108,61.0
17,SE17,"Walworth, Kennington (part), Newington",51.48764,-0.09542,41.0
8,SE8,"Deptford, Evelyn, Rotherhithe (part)",51.48114,-0.02467,27.0
13,SE13,"Lewisham, Hither Green, Ladywell",51.46196,-0.00754,25.0
16,SE16,"Rotherhithe (part), Surrey Quays, South Bermon...",51.49574,-0.05157,22.0
10,SE10,"Greenwich, Maze Hill, Greenwich Peninsula",51.48454,0.00275,19.0
4,SE4,"Brockley, Crofton Park",51.46268,-0.03558,17.0
26,SE26,"Sydenham, Crystal Palace (part)",51.42674,-0.05533,16.0
14,SE14,New Cross,51.47489,-0.04038,16.0


In [26]:
#emphasise neighbourhoods that have results to differentiate from those that have none
results['emphasis'] = results['Counts'].apply(lambda x: 0 if x==0 else x+20)

choromap = folium.Map(location=[latitude, longitude], zoom_start=12)         # create map
 
chorogeo = r'PostalDistricts1.json' # geojson file

# create a numpy array of length 6 and has linear spacing from the minium to the maximum
threshold_scale = np.linspace(results['emphasis'].min(),
                              results['emphasis'].max(),
                              6, dtype=int)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum

# generate choropleth map
choromap.choropleth(
    geo_data=chorogeo,
    data=results,
    columns=['Postcode', 'emphasis'],
    key_on='feature.properties.name',
    threshold_scale=threshold_scale,
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Venue Count'
)

for lat, lon, poi in zip(neighbourhoods['Latitude'], neighbourhoods['Longitude'], neighbourhoods['Postcode']):
    text = poi
    folium.map.Marker(
        [lat, lon],
        icon=DivIcon(
            icon_size=(100,22),
            icon_anchor=(0,0),
            html='<div style="font-size: 16pt">%s</div>' % text,
            )
        ).add_to(choromap)
    
# display map
choromap

In [17]:
rcParams['figure.figsize'] = (14,10)

llon=-4.5
ulon=-4.0
llat=55.8
ulat=55.9

VenuesList = VenuesList[(VenuesList['Venue_Lng'] > llon) & (VenuesList['Venue_Lng'] < ulon) & (VenuesList['Venue_Lat'] > llat) &(VenuesList['Venue_Lat'] < ulat)]

my_map = Basemap(projection='merc',
            resolution = 'l', area_thresh = 1000.0,
            llcrnrlon=llon, llcrnrlat=llat, #min longitude (llcrnrlon) and latitude (llcrnrlat)
            urcrnrlon=ulon, urcrnrlat=ulat) #max longitude (urcrnrlon) and latitude (urcrnrlat)

my_map.drawcoastlines()
my_map.drawcountries()
# my_map.drawmapboundary()
my_map.fillcontinents(color = 'white', alpha = 0.3)
my_map.shadedrelief()

# To collect data based on locations        
xs,ys = my_map(np.asarray(VenuesList.Venue_Lng), np.asarray(VenuesList.Venue_Lat))
VenuesList['xm']= xs.tolist()
VenuesList['ym'] =ys.tolist()

#Visualization1
for index,row in VenuesList.iterrows():
#   x,y = my_map(row.Long, row.Lat)
   my_map.plot(row.xm, row.ym,markerfacecolor =([1,0,0]),  marker='o', markersize= 5, alpha = 0.75)
#plt.text(x,y,stn)
plt.show()

NameError: name 'Basemap' is not defined

In [27]:
sklearn.utils.check_random_state(1000)
Clus_dataSet = VenuesList[['xm','ym']]
Clus_dataSet = np.nan_to_num(Clus_dataSet)
Clus_dataSet = StandardScaler().fit_transform(Clus_dataSet)

# Compute DBSCAN
db = DBSCAN(eps=0.1, min_samples=7).fit(Clus_dataSet)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_
VenuesList["Clus_Db"]=labels

# Number of clusters in labels, ignoring noise if present.
realClusterNum=len(set(labels)) - (1 if -1 in labels else 0)
clusterNum = len(set(labels)) 

print('Estimated number of clusters: %d' % realClusterNum)
print('Estimated number of noise points: %d' % clusterNum)

# A sample of clusters
VenuesList.head()

KeyError: "None of [Index(['xm', 'ym'], dtype='object')] are in the [columns]"

In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(clusterNum)
ys = [i+x+(i*x)**2 for i in range(clusterNum)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
for lat, lon, poi, cluster in zip(VenuesList['Venue_Lat'], VenuesList['Venue_Lng'], VenuesList['Venue_Name'], VenuesList['Clus_Db']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [38]:

finalmap = folium.Map(location=[latitude, longitude], zoom_start=13)         # create map

for lat, lon, poi in zip(neighbourhoods['Latitude'], neighbourhoods['Longitude'], neighbourhoods['Postcode']):
    text = poi
    folium.map.Marker(
        [lat, lon],
        icon=DivIcon(
            icon_size=(100,22),
            icon_anchor=(0,0),
            html='<div style="font-size: 18pt">%s</div>' % text,
            )
        ).add_to(finalmap)
    
chorogeo = r'PostalDistricts1.json' # geojson file

# create a numpy array of length 6 and has linear spacing from the minium total immigration to the maximum total immigration
threshold_scale = np.linspace(results['emphasis'].min(),
                              results['emphasis'].max(),
                              6, dtype=int)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration

# generate choropleth map
finalmap.choropleth(
    geo_data=chorogeo,
    data=results,
    columns=['Postcode', 'emphasis'],
    key_on='feature.properties.name',
    threshold_scale=threshold_scale,
    fill_color='PuBuGn', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Venue Count'
)

# set color scheme for the clusters
x = np.arange(clusterNum)
ys = [i+x+(i*x)**2 for i in range(clusterNum)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
for lat, lon, poi, cluster in zip(VenuesList['Venue_Lat'], VenuesList['Venue_Lng'], VenuesList['Venue_Name'], VenuesList['Clus_Db']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(finalmap)

# display map
finalmap